# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
print np.sum(student_data.passed == 'yes')
print np.sum(student_data.passed == 'no')
print np.count_nonzero(student_data.passed == 'yes')

# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!
265
130
265


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = np.size(student_data, 0)
n_features = np.size(student_data, 1)
n_passed =np.sum(student_data.passed == 'yes')
n_failed =np.sum(student_data.passed == 'no')
grad_rate = float(n_passed)/(n_students)
print float(n_passed)/ float(n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

0.670886075949
Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [93]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))


Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


AttributeError: 'str' object has no attribute 'dtype'

In [97]:
#y_all = preprocess_features(y_all)
print y_all #= y_all.replace(['yes', 'no'], [1, 0])
#print "Processed feature columns ({}):-\n{}".format(len(y_all.columns), list(y_all.columns))

0      0
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1
10     0
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     0
19     1
20     1
21     1
22     1
23     1
24     0
25     0
26     1
27     1
28     1
29     1
      ..
365    1
366    1
367    0
368    1
369    1
370    0
371    1
372    1
373    0
374    1
375    1
376    1
377    1
378    1
379    1
380    1
381    0
382    1
383    0
384    0
385    1
386    0
387    0
388    0
389    0
390    0
391    1
392    0
393    1
394    0
Name: passed, dtype: int64


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [99]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 100  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

# Shuffle Dataframe
X_all_shuf = X_all.reindex(np.random.permutation(X_all.index))
Y_all_shuf = y_all.reindex(np.random.permutation(y_all.index))

X_train = X_all_shuf[:num_train]
y_train = Y_all_shuf[:num_train]
X_test = X_all_shuf[num_train+1:num_all]
y_test = Y_all_shuf[num_train+1:num_all]
print y_train
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data
y_test

2      1
269    0
15     1
245    1
381    0
236    1
268    1
372    1
303    1
162    0
150    0
287    1
96     1
93     1
288    1
158    1
348    1
319    1
300    1
137    0
344    1
241    1
47     1
208    1
394    0
48     1
229    1
371    1
377    1
221    0
      ..
1      0
376    1
255    0
159    1
153    0
380    1
223    1
90     0
29     1
44     0
320    1
54     1
73     1
122    1
180    0
18     0
324    1
41     1
35     0
330    0
176    1
104    1
146    0
250    0
384    0
156    1
272    1
19     1
242    0
164    0
Name: passed, dtype: int64
Training set: 100 samples
Test set: 294 samples


373    0
200    1
266    1
196    1
243    1
293    1
339    1
0      0
99     0
70     1
226    1
94     1
152    1
109    1
62     0
227    1
336    1
341    0
366    1
325    1
149    1
51     1
279    1
360    1
201    1
286    1
368    1
340    1
314    1
393    1
      ..
388    0
298    1
275    1
30     1
14     1
101    1
183    0
4      1
114    0
6      1
346    1
113    1
199    1
130    0
163    1
27     1
219    1
347    0
170    0
365    1
379    1
270    0
271    1
321    0
385    1
228    0
11     1
143    1
285    1
382    1
Name: passed, dtype: int64

## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [100]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
#clf = LinearRegression()# MLPClassifier(algorithm='l-bfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1) #SVC()
from sklearn.naive_bayes import GaussianNB
#clf = GaussianNB()
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training GradientBoostingClassifier...
Done!
Training time (secs): 0.079
GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [105]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    #return f1_score(target.values, y_pred, pos_label='yes')
    return f1_score(target.values, y_pred)
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)


Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0


In [69]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.68115942029


In [70]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [71]:
# TODO: Train and predict using two other models

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [72]:
# TODO: Fine-tune your model and report the best F1 score

In [73]:
from sklearn import grid_search


In [113]:
parameters = {'max_depth':(1,2,3,4,5,6,7,8,9,10)}
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 2, 5, 7, 10]}

reg = grid_search.GridSearchCV( SVC(), parameters, verbose=10, scoring = "median_absolute_error")
train_classifier(reg, X_train, y_train)


Training GridSearchCV...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] kernel=linear, C=1 ..............................................
[CV] .................... kernel=linear, C=1, score=-1.000000 -   0.0s
[CV] kernel=linear, C=1 ..............................................
[CV] .................... kernel=linear, C=1, score=-0.000000 -   0.0s
[CV] kernel=linear, C=1 ..............................................
[CV] .................... kernel=linear, C=1, score=-1.000000 -   0.0s
[CV] kernel=rbf, C=1 .................................................
[CV] ....................... kernel=rbf, C=1, score=-0.000000 -   0.0s
[CV] kernel=rbf, C=1 .................................................
[CV] ....................... kernel=rbf, C=1, score=-0.000000 -   0.0s
[CV] kernel=rbf, C=1 .................................................
[CV] ....................... kernel=rbf, C=1, score=-0.500000 -   0.0s
[CV] kernel=linear, C=2 ......................................

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  24 tasks       | elapsed:    0.2s


[CV] ....................... kernel=rbf, C=7, score=-1.000000 -   0.0s
[CV] kernel=linear, C=10 .............................................
[CV] ................... kernel=linear, C=10, score=-1.000000 -   0.0s
[CV] kernel=linear, C=10 .............................................
[CV] ................... kernel=linear, C=10, score=-0.500000 -   0.0s
[CV] kernel=linear, C=10 .............................................
[CV] ................... kernel=linear, C=10, score=-1.000000 -   0.0s
[CV] kernel=rbf, C=10 ................................................
[CV] ...................... kernel=rbf, C=10, score=-1.000000 -   0.0s
[CV] kernel=rbf, C=10 ................................................
[CV] ...................... kernel=rbf, C=10, score=-0.000000 -   0.0s
[CV] kernel=rbf, C=10 ................................................
[CV] ...................... kernel=rbf, C=10, score=-1.000000 -   0.0s
Done!
Training time (secs): 0.288


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.3s finished


In [114]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)



Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.004
F1 score for training set: 1.0


In [117]:

train_f1_score = predict_labels(reg, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.895104895105


In [118]:
reg.best_estimator_

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [119]:
reg.best_params_

{'C': 2, 'kernel': 'rbf'}

In [120]:

train_f1_score = predict_labels(reg, X_test, y_test)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.756043956044
